In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [3]:
import torch
import pickle
import torch
import pickle

# Load the model from the pickle file
with open('/content/drive/MyDrive/Colab Notebooks/model_bert_V3.pkl', 'rb') as f:
    model_bert = pickle.load(f)


device = torch.device('cuda' if torch.cuda.is_available() else 'tpu')
model_bert = model_bert.to(device)

In [4]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import shutil
import random
import sys
import time


import re

from textblob import TextBlob
from prettytable import PrettyTable

import nltk
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import bigrams, trigrams

from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from transformers import AdamW
import torch

from sklearn.preprocessing import LabelEncoder

In [5]:
path_lyrics='/content/drive/MyDrive/Colab Notebooks/NLP_Data/lyrics_top_songs_dataset.csv' 

path_lyrics_processed='/content/drive/MyDrive/Colab Notebooks/NLP_Data/bert_preprocessed_songs.csv'



# Preprocessing the lyrics

In [ ]:
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import sent_tokenize


def preprocess(text):
    if isinstance(text, str):
        lines = text.split('\n') 
        preprocessed_lines = []
        lemmatizer = WordNetLemmatizer()  
        stop_words = set(stopwords.words('english'))  
        
        for line in lines:
            line = line.lower() 
            line = re.sub(r"[^a-zA-Z0-9_\n ]", "", line) 
            
            words = word_tokenize(line)  
            
            # Remove stop words
            filtered_words = [word for word in words if word not in stop_words]
            
            lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]  # Apply lemmatization to each word
            preprocessed_line = ' '.join(lemmatized_words)  # Join the preprocessed words
            
            preprocessed_lines.append(preprocessed_line)  
        
        return '\n'.join(preprocessed_lines)  # Join the preprocessed lines using line breaks

In [6]:
df_lyrics_processed = pd.read_csv(path_lyrics_processed)
#df_lyrics['lyrics'] = df_lyrics['lyrics'].apply(preprocess)

df_lyrics_processed.head()

,title,tag,artist,year,views,lyrics,language
0,Stand by Me,rb,Ben E. King,1960,1590384,night come\nland dark\nmoon light well see\nwo...,en
1,Georgia On My Mind,rock,Ray Charles,1960,188333,georgia\ngeorgia\nwhole day\nwhole day\nold sw...,en
2,At Last,rb,Etta James,1960,176380,last\nlove come along\nlonely day\nlife like s...,en
3,Wonderful World,pop,Sam Cooke,1960,85354,dont know much history\ndont know much biology...,en
4,Are You Lonesome Tonight?,pop,Elvis Presley,1960,72279,lonesome tonight miss tonight\nsorry drifted a...,en


# Apply the BERT model to lyrics

In [ ]:
model_bert.eval()


In [8]:
duplicates = df_lyrics_processed.duplicated(subset='lyrics', keep=False)

if duplicates.any():
    # Keep only the rows with the lowest 'year' for each duplicate lyric
    df_lyrics_processed = df_lyrics_processed.sort_values('year').drop_duplicates(subset='lyrics', keep='first')

    # Reset the index
    df_lyrics_processed = df_lyrics_processed.reset_index(drop=True)

    print("Duplicates removed")
else:
    print("No duplicates found")

Duplicates removed. Only rows with the lowest 'year' for each duplicate lyric are kept.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

phobia_lyrics = []

lyrics = df_lyrics_processed['lyrics'][50000:]
titles = df_lyrics_processed['title']


lyrics.info()

In [43]:

list_songs = [
    'Im Not Gay - J Pee',
    'Im Not a Loser - Descendents'
]

# Extract the song titles from the desired songs list
desired_titles = [song.split(" - ")[0] for song in list_songs]

# Extract the artists from the desired songs list
desired_artists = [song.split(" - ")[1] for song in list_songs]

# Filter the DataFrame based on the desired songs and artists
filtered_lyrics = df_lyrics_processed[df_lyrics_processed['title'].isin(desired_titles) & df_lyrics_processed['artist'].isin(desired_artists)]

# Print the filtered DataFrame
filtered_lyrics.head()

,title,tag,artist,year,views,lyrics,language
22068,Im Not a Loser,rock,Descendents,1982,60820,think im loser\ncause pant low\nthink im slob\...,en
53869,Im Not Gay,rap,J Pee,2013,142479,j pee baby yup yup\nyeah know\ncheck shit man\...,en


In [10]:
lyric_list = []
prob_list = []
title_list = []
line_list = []

for lyric in lyrics:

    chunk_probs = []
    max_prob = 0.0
    max_prob_line = ""
    lines = lyric.split('\n')

    for i in range(len(lines) - 1):  # Iterate over each line pair (2-line context)
      line1 = lines[i]
      line2 = lines[i + 1]

      if line1.strip() and line2.strip(): 
        chunk_text = f"{line1} {line2}"  # Combine the lines into a 2-line context

        if len(set(chunk_text.split())) >= 2:  # Check if there are two or more unique words
              encoded_input = tokenizer.encode_plus(chunk_text, truncation=True, padding=True, return_tensors='pt')  # Encode the chunk

              # Prepare input tensors
              input_ids = encoded_input['input_ids'].to(device)
              attention_mask = encoded_input['attention_mask'].to(device)

              # Perform inference
              with torch.no_grad():
                  outputs = model_bert(input_ids, attention_mask)  # Call the BERT model

              # Apply softmax
              probs = torch.softmax(outputs.logits, dim=1)

              # Get the probability for LGBTQ-phobia
              phobia_prob = probs[0][0].item()
              chunk_probs.append(phobia_prob)

              if phobia_prob > max_prob:
                  max_prob = phobia_prob
                  max_prob_line = chunk_text
        

    # Append the data to the lists
    lyric_list.append(lyric)
    prob_list.append(max_prob)
    title_list.append(titles[df_lyrics_processed['lyrics'].tolist().index(lyric)])
    line_list.append(max_prob_line)


# Create the DataFrame
df_export = pd.DataFrame({'Lyric': lyric_list, 'Title': title_list, 'Max Probability': prob_list, 'Max Probability Line': line_list})

# Sort the DataFrame by "Max Probability" column in descending order
df_export = df_export.sort_values(by='Max Probability', ascending=False)


df_export.head()


df_export.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP_Data/songs_and_probabilities_bert_V3_63000.csv', index=False)